# Character LSTM

In this notebook, I'll construct a character-level LSTM. The network will train on some text, then generate new text character by character. As an example, I will train on Anna Karenina.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Below is the general architecture of the character-wise RNN.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data
Loading the Anna Karenina text file and convert it into integers. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([39, 51, 18, 82, 61,  8, 73, 10,  2, 16, 16, 16, 41, 18, 82, 82, 37,
       10, 19, 18,  4, 17, 79, 17,  8, 44, 10, 18, 73,  8, 10, 18, 79, 79,
       10, 18, 79, 17, 38,  8, 25, 10,  8, 64,  8, 73, 37, 10, 78, 21, 51,
       18, 82, 82, 37, 10, 19, 18,  4, 17, 79, 37, 10, 17, 44, 10, 78, 21,
       51, 18, 82, 82, 37, 10, 17, 21, 10, 17, 61, 44, 10, 23,  0, 21, 16,
        0, 18, 37, 66, 16, 16, 34, 64,  8, 73, 37, 61, 51, 17, 21])

## One-hot encoding


In [26]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    
    #*arr looks inside 
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [27]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

one_hot

array([[[0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

## Making training mini-batches
We are going to cut data that doesn't fit to the integer number of minibatches that are tensors with shapes: seq_length, batch_size

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Testing

In [28]:
#test on another data
arr = np.array([1,2,3,4,1,2,3,4,1,2,3,4, 1,2,3,4])
for i in get_batches(arr, 2,3):
    print(i)

(array([[1, 2, 3],
       [3, 4, 1]]), array([[2, 3, 4],
       [4, 1, 2]]))
(array([[4, 1, 2],
       [2, 3, 4]]), array([[1, 2, 1],
       [3, 4, 3]]))


In [29]:
#test on our text data
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [30]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[39 51 18 82 61  8 73 10  2 16]
 [44 23 21 10 61 51 18 61 10 18]
 [ 8 21 59 10 23 73 10 18 10 19]
 [44 10 61 51  8 10 35 51 17  8]
 [10 44 18  0 10 51  8 73 10 61]
 [35 78 44 44 17 23 21 10 18 21]
 [10 22 21 21 18 10 51 18 59 10]
 [80 46 79 23 21 44 38 37 66 10]]

y
 [[51 18 82 61  8 73 10  2 16 16]
 [23 21 10 61 51 18 61 10 18 61]
 [21 59 10 23 73 10 18 10 19 23]
 [10 61 51  8 10 35 51 17  8 19]
 [44 18  0 10 51  8 73 10 61  8]
 [78 44 44 17 23 21 10 18 21 59]
 [22 21 21 18 10 51 18 59 10 44]
 [46 79 23 21 44 38 37 66 10 55]]


---
## Defining the network


In [31]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [32]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        
        #self.parameters() makes an iterator of the model parameters
        #next(...).data returns tensor of values of the first parameters of the model
        weight = next(self.parameters()).data 
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Training network

In [18]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model and training


In [33]:
# define and print the net
n_hidden=1024
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1024, out_features=83, bias=True)
)


In [20]:
batch_size = 128
seq_length = 16
n_epochs = 7 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/7... Step: 10... Loss: 3.1445... Val Loss: 3.1676
Epoch: 1/7... Step: 20... Loss: 3.1702... Val Loss: 3.1365


KeyboardInterrupt: 

## Checkpoint

Save the model if needed (uncomment)

In [34]:
"""
#Saving model
model_name = '7_epochs_1024hidden.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)
"""

"\n#Saving model\nmodel_name = '7_epochs_1024hidden.net'\n\ncheckpoint = {'n_hidden': net.n_hidden,\n              'n_layers': net.n_layers,\n              'state_dict': net.state_dict(),\n              'tokens': net.chars}\n\nwith open(model_name, 'wb') as f:\n    torch.save(checkpoint, f)\n"

---
## Making Predictions

Now we will feed 'prime' text to the network, than let network generate last part itself.
To make text not very similar to the train data we will **topk sample** 'top_k' digits, using probabilities according to the NN Softmax output.

In [35]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [36]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [37]:
print(sample(net, 1000, prime='eleonora', top_k=2))

eleonoragggOgOgOOgOOOOOggggOgOOgggggOgggggOggOOOOOOgOOOgOgggOOOgOggggggOggOggOggOOggggOOOOOOggggOOOggOggggOOggOggOOOOOgggggggggOggggOOggggOgOOOOOOgOOggOgOOOOOgggOggggOggOOOgOOggggOggOggggOOgggOggOgOgOgOggOggOgOgOOgOOgOOgOgOggOOOgOgOOOgOgggOOgOgggOgOOOgOOOgOOOggOOggOOOOggOgOOgOgOOOOgOOgggOggOggOOgOOgOggOOgOggOOOOgOgOOOgggOgOOOOOOgOOgOgOOOOOOgOOgOOOOgOgOgggOgOggOgggOOgggggOgOgggOOgggOgOgOgOgggOgggggggOgOOOgOOgOgggOOgOOggOgOOgggOOOOgOOgOggOOOOgOOOOgggOgOggOgOgOOggOgggOgggggggOOggOggggOggOgggOgOOgOgOggOgOOggOgOgOOOOOOggggggOgOgOOggggOgOgOgggOOgggggOgOgOgggOOOggOOgggOggOgOgOOOOOOOOgOgggOgggggOOOgggOgOOggOOggOggOOOgOgggOggggggOOgOOggOOOOOggOOOgOOgOggOgOggOOggOgOgOOgOOgOOgOggggggOOOggOgOgOOgOOgOggOOgOgOOOgggggOggOggOgOOgOgOOgOggOgOgOggOOgOOgOggOOOOgOOggOgggOOgOOgOOOgggOOOgOgOgOOggOgOOgOgggOgOOggOOgOOgOgOOOgOgOOOOggggOgOgOOOOggOgOgggOgOgOgOOOOggOOOOggOggggOgOgOOOgOOgOOOOOgOOOOOOggOgOOOOgOOOggOOOgOgOOgggOOgOOOgggggOOOgOggggOOOgggOOOgOOOggOgOggOgOOggOggOggggOOOgOggOOOgggOggOgOOOggOOOOgOOggggOggO

## Loading a checkpoint

In [38]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('7_epochs_1024hidden.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [41]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="that was"))

that was to be sure and decided than it
has been satisfied with whem he said to her.

"To do it. A stating or this peasant of
me.
In whom I was in the life is already in secretary of my sister's with a studion," answered Anna, looking at him as she caught
sight, and he wanted to see him all about all the world, she was silent. He had been staring them as
to show it was at
once that he should be a great thing sent a country was the
fundling thoughts they could not speak about, was sitting down. "Why is it the passage
alone today? Wime, to say to you," she said,
smiling.

"Well, well, the land, the peasants. Would I do."

"Yes; you do not say so. With the patient shelp to the least, and then what were the most important to min your heart to be sole of you," said the lady, the stream of his lover,
a creak to her.

"Then
you must said these president, in their memories of the carriage who's to be terrible to you. I am not told, and told me a fine first sense of charming sincere. A minute i